# Assignment
For this assigment, we are going to do something like k-fold validation to determine the true expected performance of our model

The reason this is a little tricky is due to the fact that we would like to our test and train samples to always have different users.   As we noted in the **prep** workbook, every 4 users is about 10% of the sample, so lets use this to divide our data and determine our average performance of our model.

**TASK**:
To make this concrete, I want you to divide the full sample up into 9 folds, like this:
* fold1: test sample is users >=0 and <4; train sample is users>=4
* fold2: test sample is users >=4 and <8; train sample is users<4 or users>=8
...etc...

Determine the performance of the CNN version of the model (use the model made using the Keras Functional API) for each fold, then average the results.

**EXTRA**: Incorporate a multi-head model (with at least 3 heads) each using a different kernel size.  Do the averaging as above.   Can you come up with hyperparamters that beat the performance of the earlier 2-headed model (though that was measured with just a single fold).

The assigment below has some starter code in to help you begin.


# Code to read data in and normalize it.

In [2]:
import pandas as pd
import numpy as np
#
# Use this to convert text to floating point
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

column_names = ['user-id',
                    'activity',
                    'timestamp',
                    'x-axis',
                    'y-axis',
                    'z-axis']
df = pd.read_csv('/fs/scratch/PAS1495/physics6820/WISDM/WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt',
                     header=None,
                     names=column_names)

# Last column has a ";" character which must be removed ...
df['z-axis'].replace(regex=True,
      inplace=True,
      to_replace=r';',
      value=r'')
    # ... and then this column must be transformed to float explicitly
df['z-axis'] = df['z-axis'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
#
# Get rid if rows wth missing data
df.dropna(axis=0, how='any', inplace=True)

from sklearn import preprocessing
# Define column name of the label vector
LABEL = 'ActivityEncoded'
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df[LABEL] = le.fit_transform(df['activity'].values.ravel())

#
# Normalize the data: to make things simple, just normalize all of the data (pre train/test) by 20
max_all = 20.0
df['x-axis'] = df['x-axis'] / 20.0
df['y-axis'] = df['y-axis'] / 20.0
df['z-axis'] = df['z-axis'] / 20.0

print(df.head())

max_x = df['x-axis'].max()
max_y = df['y-axis'].max()
max_z = df['z-axis'].max()

print("max values ", max_x,max_y,max_z)


   user-id activity       timestamp    x-axis    y-axis    z-axis  \
0       33  Jogging  49105962326000 -0.034732  0.634027  0.025198   
1       33  Jogging  49106062271000  0.250614  0.563201  0.047671   
2       33  Jogging  49106112167000  0.245166  0.544133 -0.004086   
3       33  Jogging  49106222305000 -0.030646  0.924822  0.151186   
4       33  Jogging  49106332290000 -0.059249  0.605424  0.360258   

   ActivityEncoded  
0                1  
1                1  
2                1  
3                1  
4                1  
max values  0.9974999999999999 1.002 0.9804999999999999


In [3]:
from scipy import stats

# Same labels will be reused throughout the program
LABELS = ['Downstairs',
          'Jogging',
          'Sitting',
          'Standing',
          'Upstairs',
          'Walking']
# The number of steps within one time segment
TIME_PERIODS = 80    # since there are 50 measurements/sec, this is 1.6 seconds of data
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE_TRAIN = 40
STEP_DISTANCE_TEST = 80

def create_segments_and_labels(df, time_steps, step, label_name):

    # x, y, z acceleration as features
    N_FEATURES = 3
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['x-axis'].values[i: i + time_steps]
        ys = df['y-axis'].values[i: i + time_steps]
        zs = df['z-axis'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        label = stats.mode(df[label_name][i: i + time_steps])[0][0] #stats.mode finds the most common values that occurs in the array
        #The default axis is 0
        
        segments.append([xs, ys, zs]) #This segments is a list of lists (nx3). xs, ys, zs are the arrays.
        labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels

In [4]:
def shuffle_weights(model, weights=None):
    """Randomly permute the weights in `model`, or the given `weights`.
    This is a fast approximation of re-initializing the weights of a model.
    Assumes weights are distributed independently of the dimensions of the weight tensors
      (i.e., the weights have the same distribution along each dimension).
    :param Model model: Modify the weights of the given model.
    :param list(ndarray) weights: The model's weights will be replaced by a random permutation of these weights.
      If `None`, permute the model's current weights.
    """
    if weights is None:
        weights = model.get_weights()
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
    # Faster, but less random: only permutes along the first dimension
    # weights = [np.random.permutation(w) for w in weights]
    model.set_weights(weights)

In [5]:
import keras
from keras.layers import Input,Conv1D, MaxPooling1D,GlobalAveragePooling1D,Dropout,Dense
from keras.models import Model

num_classes = 6 
# Our first layer gets the input from our samples - this is 80 time steps by 3 channels
#model_m.add(Conv1D(100, 10, activation='relu', input_shape=(80,3)))
inputs1 = Input(shape=(80,3))
conv1 = Conv1D(100, 10, activation='relu')(inputs1)
#
# Anoth convolutional layer
#model_m.add(Conv1D(100, 10, activation='relu'))
conv2 = Conv1D(100, 10, activation='relu')(conv1)
#
# Max pooling 
#model_m.add(MaxPooling1D(3))
pool1 = MaxPooling1D(3)(conv2)
#
# Two more convolutional layers
#model_m.add(Conv1D(160, 10, activation='relu'))
#model_m.add(Conv1D(160, 10, activation='relu'))
conv3 = Conv1D(160, 10, activation='relu')(pool1)
conv4 = Conv1D(160, 10, activation='relu')(conv3)
#
# Global average pooling use this instead of "Flatten" - it helps reduce overfitting
#model_m.add(GlobalAveragePooling1D())
glob1 = GlobalAveragePooling1D()(conv4)
#
drop1 = Dropout(0.5)(glob1)
outputs = Dense(num_classes, activation='softmax')(drop1)

Using TensorFlow backend.


# TASK: 

Divide the full sample up into 9 folds, like this:

*  fold1: test sample is users >=0 and <4; train sample is users>=4
*  fold2: test sample is users >=4 and <8; train sample is users<4 or users>=8 
*  ...etc...

Determine the performance of the CNN version of the model (use the model made using the Keras Functional API) for each fold, then average the results.

# TASK: 
Create data with the right format

In [6]:
TIME_PERIODS = 80
STEP_DISTANCE_TRAIN = 40
STEP_DISTANCE_TEST = 80

x_train_list = []
train_onehot_list = []
x_test_list = []
test_onehot_list = []
kfold = 6

for user_groups in range(kfold):
    user_start = int(user_groups * (36/kfold) + 1)
    user_end = int(user_start + 36 / kfold)
    
    test_id = [i for i in range(user_start, user_end)]
    train_id = [i for i in range(0, user_start)]
    for i in range(user_end, 37):
        train_id.append(i)
    
    print("Processing the ", user_groups, 'th validation dataset')
    print("The test users are:\n", test_id)
    print("The training users are:\n", train_id)
    
    df_train = pd.DataFrame()
    df_test = pd.DataFrame()
    for i in train_id:
        df_train = pd.concat([df_train, df[df['user-id'] == i]])
        
    for i in test_id:
        df_test = pd.concat([df_test, df[df['user-id'] == i]])
        
    print("Training sample size", df_train.shape, "Test sample size", df_test.shape)
    #def create_segments_and_labels(df, time_steps, step, label_name):
    x_train, y_train = create_segments_and_labels(df_train, TIME_PERIODS, STEP_DISTANCE_TRAIN, LABEL)
    x_test, y_test = create_segments_and_labels(df_train, TIME_PERIODS, STEP_DISTANCE_TEST, LABEL)
    
    train_onehot = np.zeros((y_train.shape[0], 6))
    test_onehot = np.zeros((y_test.shape[0], 6))
    for i in range(len(y_train)):
        train_onehot[i][y_train[i]] = 1
        
    for i in range(len(y_test)):
        test_onehot[i][y_test[i]] = 1
        
    x_train_list.append(x_train)
    x_test_list.append(x_test)
    train_onehot_list.append(train_onehot)
    test_onehot_list.append(test_onehot)

Processing the  0 th validation dataset
The test users are:
 [1, 2, 3, 4, 5, 6]
The training users are:
 [0, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
Training sample size (939955, 7) Test sample size (158248, 7)
Processing the  1 th validation dataset
The test users are:
 [7, 8, 9, 10, 11, 12]
The training users are:
 [0, 1, 2, 3, 4, 5, 6, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
Training sample size (913943, 7) Test sample size (184260, 7)
Processing the  2 th validation dataset
The test users are:
 [13, 14, 15, 16, 17, 18]
The training users are:
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]
Training sample size (919463, 7) Test sample size (178740, 7)
Processing the  3 th validation dataset
The test users are:
 [19, 20, 21, 22, 23, 24]
The training users are:
 [0, 1, 2, 3, 4, 5, 6, 7, 

In [7]:
BATCH_SIZE = 400
EPOCHS = 50
patience = 4

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)
    ]

accuracy = 0
#Fit the model
model_m = Model(inputs=inputs1, outputs = outputs)
model_m.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

for user_groups in range(kfold):
    x_train = x_train_list[user_groups]
    x_test = x_test_list[user_groups]
    y_train_hot = train_onehot_list[user_groups]
    y_test_hot = test_onehot_list[user_groups]
    history = model_m.fit(x_train,
                  y_train_hot,
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS,
                  callbacks=callbacks_list,
                  validation_data=(x_test, y_test_hot),
                  verbose=1)

    best_val_acc =  history.history['val_acc'][-(patience+1)]
    print("Best validation accuracy is:",best_val_acc)
    accuracy += best_val_acc
    #When the fitting is done, reset the weights for the next round.
    shuffle_weights(model_m)
    # We want our users to 

accuracy = accuracy / kfold        
print("The average accuracy(expected value for our final model)")

Train on 23497 samples, validate on 11749 samples
Epoch 1/50
23497/23497 [==============================] - 56s 2ms/step - loss: 0.8142 - acc: 0.6995 - val_loss: 0.4968 - val_acc: 0.8150
Epoch 2/50
23497/23497 [==============================] - 55s 2ms/step - loss: 0.4684 - acc: 0.8284 - val_loss: 0.3851 - val_acc: 0.8488
Epoch 3/50
23497/23497 [==============================] - 55s 2ms/step - loss: 0.3653 - acc: 0.8643 - val_loss: 0.3019 - val_acc: 0.8850
Epoch 4/50
23497/23497 [==============================] - 56s 2ms/step - loss: 0.2969 - acc: 0.8940 - val_loss: 0.2415 - val_acc: 0.9111
Epoch 5/50
23497/23497 [==============================] - 55s 2ms/step - loss: 0.2525 - acc: 0.9146 - val_loss: 0.2503 - val_acc: 0.9083
Epoch 6/50
23497/23497 [==============================] - 55s 2ms/step - loss: 0.2307 - acc: 0.9243 - val_loss: 0.1844 - val_acc: 0.9396
Epoch 7/50
23497/23497 [==============================] - 57s 2ms/step - loss: 0.1923 - acc: 0.9400 - val_loss: 0.1530 - val_acc

22847/22847 [==============================] - 53s 2ms/step - loss: 0.0159 - acc: 0.9958 - val_loss: 0.0060 - val_acc: 0.9989
Epoch 25/50
22847/22847 [==============================] - 53s 2ms/step - loss: 0.0112 - acc: 0.9979 - val_loss: 0.0052 - val_acc: 0.9989
Epoch 26/50
22847/22847 [==============================] - 53s 2ms/step - loss: 0.0133 - acc: 0.9962 - val_loss: 0.0101 - val_acc: 0.9971
Epoch 27/50
22847/22847 [==============================] - 50s 2ms/step - loss: 0.0108 - acc: 0.9972 - val_loss: 0.0037 - val_acc: 0.9994
Epoch 28/50
22847/22847 [==============================] - 53s 2ms/step - loss: 0.0097 - acc: 0.9977 - val_loss: 0.0060 - val_acc: 0.9986
Epoch 29/50
22847/22847 [==============================] - 51s 2ms/step - loss: 0.0102 - acc: 0.9975 - val_loss: 0.0103 - val_acc: 0.9959
Epoch 30/50
22847/22847 [==============================] - 51s 2ms/step - loss: 0.0098 - acc: 0.9976 - val_loss: 0.0037 - val_acc: 0.9993
Epoch 31/50
22847/22847 [=====================

22174/22174 [==============================] - 51s 2ms/step - loss: 0.0244 - acc: 0.9928 - val_loss: 0.0136 - val_acc: 0.9968
Epoch 16/50
22174/22174 [==============================] - 50s 2ms/step - loss: 0.0219 - acc: 0.9939 - val_loss: 0.0175 - val_acc: 0.9937
Epoch 17/50
22174/22174 [==============================] - 51s 2ms/step - loss: 0.0305 - acc: 0.9892 - val_loss: 0.0109 - val_acc: 0.9976
Epoch 18/50
22174/22174 [==============================] - 51s 2ms/step - loss: 0.0199 - acc: 0.9939 - val_loss: 0.0074 - val_acc: 0.9986
Epoch 19/50
22174/22174 [==============================] - 50s 2ms/step - loss: 0.0137 - acc: 0.9963 - val_loss: 0.0071 - val_acc: 0.9984
Epoch 20/50
22174/22174 [==============================] - 51s 2ms/step - loss: 0.0118 - acc: 0.9968 - val_loss: 0.0052 - val_acc: 0.9987
Epoch 21/50
22174/22174 [==============================] - 50s 2ms/step - loss: 0.0075 - acc: 0.9981 - val_loss: 0.0036 - val_acc: 0.9995
Epoch 22/50
22174/22174 [=====================

22457/22457 [==============================] - 51s 2ms/step - loss: 0.0155 - acc: 0.9953 - val_loss: 0.0095 - val_acc: 0.9982
Epoch 15/50
22457/22457 [==============================] - 51s 2ms/step - loss: 0.0132 - acc: 0.9964 - val_loss: 0.0080 - val_acc: 0.9981
Epoch 16/50
22457/22457 [==============================] - 51s 2ms/step - loss: 0.0113 - acc: 0.9971 - val_loss: 0.0084 - val_acc: 0.9973
Epoch 17/50
22457/22457 [==============================] - 51s 2ms/step - loss: 0.0120 - acc: 0.9966 - val_loss: 0.0051 - val_acc: 0.9992
Epoch 18/50
22457/22457 [==============================] - 51s 2ms/step - loss: 0.0087 - acc: 0.9979 - val_loss: 0.0032 - val_acc: 0.9996
Epoch 19/50
22457/22457 [==============================] - 51s 2ms/step - loss: 0.0138 - acc: 0.9960 - val_loss: 0.0049 - val_acc: 0.9991
Epoch 20/50
22457/22457 [==============================] - 51s 2ms/step - loss: 0.0136 - acc: 0.9957 - val_loss: 0.0044 - val_acc: 0.9994
Epoch 21/50
22457/22457 [=====================

# EXTRA
Incorporate a multi-head model (with at least 3 heads) each using a different kernel size.  Do the averaging as above.   Can you come up with hyperparamters that beat the performance of the earlier 2-headed model (though that was measured with just a single fold).


Create the 3-headed network.

In [8]:
from keras.layers import Input,Conv1D, MaxPooling1D,GlobalAveragePooling1D
from keras.models import Model
from keras.layers.merge import concatenate

inputs1 = Input(shape=(80, 3))
h1conv1 = Conv1D(filters = 100, kernel_size=10, activation='relu')(inputs1)
h1conv2 = Conv1D(filters=100, kernel_size=10, activation='relu')(h1conv1)
h1pool1 = MaxPooling1D(3)(h1conv2)
h1glob1 = GlobalAveragePooling1D()(h1pool1)
h1drop1 = Dropout(0.5)(h1glob1)

inputs2 = Input(shape=(80,3))
h2conv1 = Conv1D(filters=100, kernel_size=13, activation='relu')(inputs2)
h2conv2 = Conv1D(filters=100, kernel_size=13, activation='relu')(h2conv1)
h2pool1 = MaxPooling1D(3)(h2conv2)
h2glob1 = GlobalAveragePooling1D()(h2pool1)
h2drop1 = Dropout(0.5)(h2glob1)

inputs3 = Input(shape=(80, 3))
h3conv1 = Conv1D(filters=100, kernel_size=16, activation='relu')(inputs3)
h3conv2 = Conv1D(filters=100, kernel_size=16, activation='relu')(inputs3)
h3pool1 = MaxPooling1D(3)(h3conv2)
h3glob1 = GlobalAveragePooling1D()(h3pool1)
h3drop1 = Dropout(0.5)(h3glob1)

merged = concatenate([h1drop1, h2drop1, h3drop1])

dense1 = Dense(100, activation='relu')(merged)
outputs = Dense(6, activation='softmax')(dense1)

In [9]:
BATCH_SIZE = 400
EPOCHS = 50
patience = 4

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)]

accuracy = 0
#Fit the model
model_hydra = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
model_hydra.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
for user_groups in range(kfold):
    x_train = x_train_list[user_groups]
    x_test = x_test_list[user_groups]
    y_train_hot = train_onehot_list[user_groups]
    y_test_hot = test_onehot_list[user_groups]
    history = model_hydra.fit([x_train, x_train, x_train],
                  y_train_hot,
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS,
                  callbacks=callbacks_list,
                  validation_data=([x_test, x_test, x_test], y_test_hot))

    best_val_acc =  history.history['val_acc'][-(patience+1)]
    print("Best validation accuracy is:",best_val_acc)
    accuracy += best_val_acc
    #When the fitting is done, reset the weights for the next round.
    shuffle_weights(model_m)
    # We want our users to 

accuracy = accuracy / kfold
print("The average accuracy(expected value for our final model)")

Train on 23497 samples, validate on 11749 samples
Epoch 1/50
23497/23497 [==============================] - 87s 4ms/step - loss: 0.9746 - acc: 0.6314 - val_loss: 0.6981 - val_acc: 0.6994
Epoch 2/50
23497/23497 [==============================] - 88s 4ms/step - loss: 0.6313 - acc: 0.7589 - val_loss: 0.5357 - val_acc: 0.7830
Epoch 3/50
23497/23497 [==============================] - 87s 4ms/step - loss: 0.5464 - acc: 0.7950 - val_loss: 0.4917 - val_acc: 0.8167
Epoch 4/50
23497/23497 [==============================] - 87s 4ms/step - loss: 0.5055 - acc: 0.8093 - val_loss: 0.4637 - val_acc: 0.8205
Epoch 5/50
23497/23497 [==============================] - 87s 4ms/step - loss: 0.4671 - acc: 0.8242 - val_loss: 0.4059 - val_acc: 0.8475
Epoch 6/50
23497/23497 [==============================] - 88s 4ms/step - loss: 0.4260 - acc: 0.8399 - val_loss: 0.3679 - val_acc: 0.8631
Epoch 7/50
23497/23497 [==============================] - 87s 4ms/step - loss: 0.3936 - acc: 0.8531 - val_loss: 0.3391 - val_acc

22847/22847 [==============================] - 85s 4ms/step - loss: 0.1133 - acc: 0.9624 - val_loss: 0.0825 - val_acc: 0.9777
Epoch 10/50
22847/22847 [==============================] - 84s 4ms/step - loss: 0.1057 - acc: 0.9655 - val_loss: 0.0812 - val_acc: 0.9749
Epoch 11/50
22847/22847 [==============================] - 84s 4ms/step - loss: 0.1068 - acc: 0.9644 - val_loss: 0.0746 - val_acc: 0.9781
Epoch 12/50
22847/22847 [==============================] - 84s 4ms/step - loss: 0.1014 - acc: 0.9672 - val_loss: 0.0773 - val_acc: 0.9753
Epoch 13/50
22847/22847 [==============================] - 85s 4ms/step - loss: 0.0966 - acc: 0.9691 - val_loss: 0.0710 - val_acc: 0.9802
Epoch 14/50
22847/22847 [==============================] - 84s 4ms/step - loss: 0.0946 - acc: 0.9690 - val_loss: 0.0715 - val_acc: 0.9786
Epoch 15/50
22847/22847 [==============================] - 85s 4ms/step - loss: 0.0954 - acc: 0.9682 - val_loss: 0.0653 - val_acc: 0.9814
Epoch 16/50
22847/22847 [=====================

Epoch 18/50
22985/22985 [==============================] - 86s 4ms/step - loss: 0.0327 - acc: 0.9893 - val_loss: 0.0129 - val_acc: 0.9967
Epoch 19/50
22985/22985 [==============================] - 84s 4ms/step - loss: 0.0298 - acc: 0.9904 - val_loss: 0.0122 - val_acc: 0.9977
Epoch 20/50
22985/22985 [==============================] - 86s 4ms/step - loss: 0.0302 - acc: 0.9903 - val_loss: 0.0109 - val_acc: 0.9974
Epoch 21/50
22985/22985 [==============================] - 84s 4ms/step - loss: 0.0291 - acc: 0.9906 - val_loss: 0.0116 - val_acc: 0.9968
Epoch 22/50
22985/22985 [==============================] - 86s 4ms/step - loss: 0.0284 - acc: 0.9910 - val_loss: 0.0089 - val_acc: 0.9978
Epoch 23/50
22985/22985 [==============================] - 85s 4ms/step - loss: 0.0283 - acc: 0.9906 - val_loss: 0.0092 - val_acc: 0.9977
Epoch 24/50
22985/22985 [==============================] - 86s 4ms/step - loss: 0.0290 - acc: 0.9909 - val_loss: 0.0083 - val_acc: 0.9982
Epoch 25/50
22985/22985 [=========

23305/23305 [==============================] - 87s 4ms/step - loss: 0.0232 - acc: 0.9924 - val_loss: 0.0069 - val_acc: 0.9985
Epoch 19/50
23305/23305 [==============================] - 86s 4ms/step - loss: 0.0218 - acc: 0.9934 - val_loss: 0.0083 - val_acc: 0.9981
Epoch 20/50
23305/23305 [==============================] - 87s 4ms/step - loss: 0.0224 - acc: 0.9925 - val_loss: 0.0078 - val_acc: 0.9986
Epoch 21/50
23305/23305 [==============================] - 88s 4ms/step - loss: 0.0237 - acc: 0.9918 - val_loss: 0.0066 - val_acc: 0.9988
Epoch 22/50
23305/23305 [==============================] - 87s 4ms/step - loss: 0.0199 - acc: 0.9939 - val_loss: 0.0068 - val_acc: 0.9990
Epoch 23/50
23305/23305 [==============================] - 85s 4ms/step - loss: 0.0199 - acc: 0.9934 - val_loss: 0.0066 - val_acc: 0.9990
Epoch 24/50
23305/23305 [==============================] - 87s 4ms/step - loss: 0.0211 - acc: 0.9928 - val_loss: 0.0064 - val_acc: 0.9991
Epoch 25/50
23305/23305 [=====================

Epoch 43/50
22457/22457 [==============================] - 72s 3ms/step - loss: 0.0143 - acc: 0.9954 - val_loss: 0.0036 - val_acc: 0.9993
Epoch 44/50
22457/22457 [==============================] - 73s 3ms/step - loss: 0.0119 - acc: 0.9964 - val_loss: 0.0032 - val_acc: 0.9994
Epoch 45/50
22457/22457 [==============================] - 72s 3ms/step - loss: 0.0142 - acc: 0.9958 - val_loss: 0.0036 - val_acc: 0.9995
Best validation accuracy is: 0.9993766147186499
The average accuracy(expected value for our final model)


The average accuracy is very good: 0.99934.
Very good result and way better than the 2-headed model